In [1]:
import os
import sys
import glob
from operator import index
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from traitsui.examples.demo.Advanced.Tabular_editor_with_context_menu_demo import columns

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')

csv_ts_error = os.path.join(root_stat, '01_continuous', '01_results', 'omc_mmc_error.csv')
df_ts_error = pd.read_csv(csv_ts_error, sep=';')

csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')

df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s = idx_s_multicam + idx_s_singlecam

C:\Users\johan\AppData\Local\Temp\ipykernel_22576\303231503.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ts_error = pd.read_csv(csv_ts_error, sep=';')


In [8]:
df_ts_error = df_ts_error[(df_ts_error['id_s'].isin(idx_s_reduced)) & (~df_ts_error['id_p'].isin(ignore_id_p)) &
                          (df_ts_error['dynamic'] == 'fixed') & (df_ts_error['normalized'] == 'original')]

In [15]:
df_ts_error_trial = df_ts_error[pd.notna(df_ts_error['id_t'])]
df_ts_error_patient = df_ts_error[(pd.isna(df_ts_error['id_t'])) & (pd.notna(df_ts_error['id_p']))]
df_ts_error_setting = df_ts_error[(pd.isna(df_ts_error['id_t'])) & (pd.isna(df_ts_error['id_p']))]

In [12]:
df_ts_error_trial

,id,id_s,id_p,id_t,condition,dynamic,normalized,metric,mean,median,std,rmse,rmse_std
0,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,hand_vel,-2.474857,-3.681392,33.687505,33.778291,24.870351
1,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,elbow_vel,0.170871,0.038212,0.452060,0.483275,0.331168
2,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,trunk_disp,0.082558,-0.216019,1.857228,1.859062,1.081438
3,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,trunk_ang,-64.125089,-64.681983,64.189987,90.732472,62.924541
4,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,elbow_flex_pos,2.186955,2.038520,4.075282,4.625007,2.576745
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255586,S018_P252_T083,S018,P252,T083,affected,fixed,original,trunk_disp,2.558258,0.832685,11.496595,11.777792,6.272443
255587,S018_P252_T083,S018,P252,T083,affected,fixed,original,trunk_ang,-37.722361,-37.954190,37.727432,53.351061,37.456293
255588,S018_P252_T083,S018,P252,T083,affected,fixed,original,elbow_flex_pos,-0.593447,0.607798,3.326452,3.378973,1.908764
255589,S018_P252_T083,S018,P252,T083,affected,fixed,original,shoulder_flex_pos,-6.795023,-6.627187,7.631763,10.218422,5.959129


In [13]:
df_ts_error

,id,id_s,id_p,id_t,condition,dynamic,normalized,metric,mean,median,std,rmse,rmse_std
0,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,hand_vel,-2.474857,-3.681392,33.687505,33.778291,24.870351
1,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,elbow_vel,0.170871,0.038212,0.452060,0.483275,0.331168
2,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,trunk_disp,0.082558,-0.216019,1.857228,1.859062,1.081438
3,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,trunk_ang,-64.125089,-64.681983,64.189987,90.732472,62.924541
4,S001_P07_T002,S001,P07,T002,unaffected,fixed,original,elbow_flex_pos,2.186955,2.038520,4.075282,4.625007,2.576745
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262509,S018_P252,S018,P252,NaN,unaffected,fixed,original,trunk_disp,1.354654,0.829535,5.562869,5.863083,3.658686
262510,S018_P252,S018,P252,NaN,unaffected,fixed,original,trunk_ang,-37.496571,-60.277558,37.433992,52.984080,37.176441
262511,S018_P252,S018,P252,NaN,unaffected,fixed,original,elbow_flex_pos,-6.000685,-5.077928,8.964507,10.821943,6.422028
262512,S018_P252,S018,P252,NaN,unaffected,fixed,original,shoulder_flex_pos,-3.596865,-1.770773,8.955739,9.668777,6.430893
